<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importar-Librerías" data-toc-modified-id="Importar-Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importar Librerías</a></span></li><li><span><a href="#Funciones-y-Clases" data-toc-modified-id="Funciones-y-Clases-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Funciones y Clases</a></span></li><li><span><a href="#Preprocesamiento-de-Datos" data-toc-modified-id="Preprocesamiento-de-Datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocesamiento de Datos</a></span><ul class="toc-item"><li><span><a href="#Construcción-del-Dataset-de-Productos" data-toc-modified-id="Construcción-del-Dataset-de-Productos-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Construcción del Dataset de Productos</a></span><ul class="toc-item"><li><span><a href="#Lectura-de-los-Datos" data-toc-modified-id="Lectura-de-los-Datos-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Lectura de los Datos</a></span></li><li><span><a href="#Transformación-de-los-Datos" data-toc-modified-id="Transformación-de-los-Datos-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Transformación de los Datos</a></span></li></ul></li><li><span><a href="#Selección-de-Predictores" data-toc-modified-id="Selección-de-Predictores-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Selección de Predictores</a></span></li><li><span><a href="#Estandarización,-OneHotEncoding-y-Train-Test-Split" data-toc-modified-id="Estandarización,-OneHotEncoding-y-Train-Test-Split-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Estandarización, OneHotEncoding y Train-Test Split</a></span></li></ul></li><li><span><a href="#Modelos-de-Predicción---Sold-Quantity" data-toc-modified-id="Modelos-de-Predicción---Sold-Quantity-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modelos de Predicción - Sold Quantity</a></span><ul class="toc-item"><li><span><a href="#XGBoost-(Gradient-Boosting-Trees)" data-toc-modified-id="XGBoost-(Gradient-Boosting-Trees)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>XGBoost (Gradient Boosting Trees)</a></span></li><li><span><a href="#NNs-(Redes-Neuronales)" data-toc-modified-id="NNs-(Redes-Neuronales)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>NNs (Redes Neuronales)</a></span></li><li><span><a href="#Random-Forest-(Bosques-Aleatorios)" data-toc-modified-id="Random-Forest-(Bosques-Aleatorios)-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Random Forest (Bosques Aleatorios)</a></span></li><li><span><a href="#Stacked-Model" data-toc-modified-id="Stacked-Model-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Stacked Model</a></span></li></ul></li><li><span><a href="#Alternativa:-Modelos-de-Clasificación" data-toc-modified-id="Alternativa:-Modelos-de-Clasificación-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Alternativa: Modelos de Clasificación</a></span><ul class="toc-item"><li><span><a href="#Definición-de-los-Rangos-de-Cantidades-Vendidas" data-toc-modified-id="Definición-de-los-Rangos-de-Cantidades-Vendidas-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Definición de los Rangos de Cantidades Vendidas</a></span></li><li><span><a href="#Neural-Network" data-toc-modified-id="Neural-Network-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Neural Network</a></span></li></ul></li><li><span><a href="#Conclusiones-y-Reflexiones" data-toc-modified-id="Conclusiones-y-Reflexiones-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusiones y Reflexiones</a></span></li></ul></div>

# Sold Quantity Prediction

El propósito de este notebook es construir una serie de modelos de predicción para estimar la variable `ITEM_SOLD_QUANTITY` de un item de [Mercado Libre](https://www.mercadolibre.com.co/) con el dataset construido en [`MeliMLChallenge/Notebook/ApiPullingData.ipynb`](https://github.com/juanse1608/MeliMLChallenge/blob/main/Notebooks/ApiPullingData.ipynb).

Se recuerda que el dataset consta de __14574__ items (tratanto de tener la misma cantidad para cada una de las categorías de ítems de Meli Colombia) y __50__ variables asociadas al ítem, su vendedor y su envío.

## Importar Librerías

In [762]:
# Cambia el ancho de las celdas
from IPython.display import HTML, display
display(HTML(data="""<style>div#notebook-container{width: 60%;}div#menubar-container{width: 65%;} div#maintoolbar-container{width: 99%;}</style>"""))

In [874]:
# Procesamiento de datos
import os
import re
import pandas as pd
import numpy as np
import datetime as dt
import itertools as it
import requests

# Visualización
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib.ticker import PercentFormatter
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# ML y DS
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
import tensorflow as tf

## Funciones y Clases

Esta sección tiene las funciones y clases creadas que se usarán a los largo de este notebook.

In [764]:
# Funcion que en encuentra el valor mas cercano de un array a un valor 
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

## Preprocesamiento de Datos

En esta sección se realiza un proceso general de limpieza y organización de los datos previo a la exploración de los datos.

### Construcción del Dataset de Productos

#### Lectura de los Datos
 
En esta subsección se leen el dataset de productos, se imprimen sus dimensiones, sus columnas y se mencionan sus tipos de variables.

In [765]:
# Se lee el .csv con el datset construido
products = pd.read_csv('../Data/Datasets/PRODUCTOS.csv')
products.columns = products.columns.str.upper() # Pone los nombres de las columnas en mayusculas
print('LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: {}'.format(products.shape))

LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: (14574, 50)


In [766]:
# Printea la columnas del dataset
products.columns

Index(['ITEM_ID', 'ITEM_SITE_ID', 'ITEM_TITLE', 'ITEM_PRICE',
       'ITEM_SALE_PRICE', 'ITEM_CURRENCY_ID', 'ITEM_AVAILABLE_QUANTITY',
       'ITEM_SOLD_QUANTITY', 'ITEM_BUYING_MODE', 'ITEM_LISTING_TYPE_ID',
       'ITEM_STOP_TIME', 'ITEM_CONDITION', 'ITEM_PERMALINK', 'ITEM_THUMBNAIL',
       'ITEM_THUMBNAIL_ID', 'ITEM_ACCEPTS_MERCADOPAGO', 'ITEM_ORIGINAL_PRICE',
       'ITEM_CATEGORY_ID', 'ITEM_OFFICIAL_STORE_ID', 'ITEM_DOMAIN_ID',
       'ITEM_CATALOG_PRODUCT_ID', 'ITEM_ORDER_BACKEND',
       'ITEM_USE_THUMBNAIL_ID', 'SEARCH_CATEGORY_ID', 'SEARCH_CATEGORY_NAME',
       'SEARCH_OFFSET', 'SELLER_ID', 'SELLER_REP_TRANSACTIONS_TOTAL',
       'SELLER_REP_TRANSACTIONS_CANCELED', 'SELLER_REP_RATING_NEG',
       'SELLER_REP_RATING_POS', 'SELLER_REP_RATING_NEU',
       'SELLER_TRANSACTIONS_COMPLETED', 'SELLER_STATUS',
       'SELLER_METRICS_CLAIMS_RATE', 'SELLER_METRICS_CLAIMS_VALUE',
       'SELLER_METRICS_CLAIMS_PERIOD', 'SELLER_METRICS_DELAY_RATE',
       'SELLER_METRICS_DELAY_VALUE', 'SEL

Se puede ver claramente que se encuentran los siguientes tipos de variables:
    
1. Tipo `ITEM`: Variables asociadas al ítem.


2. Tipo `SELLER`: Variables asociadas al vendedor.


3. Tipo `SHIPPING`: Variables asociadas al envío.


4. Tipo `ADRESS`: Variable que indica el departamento donde se encuentra el ítem.


5. Tipo `SEARCH`: Variables asociadas a la búsqueda: categoría y offset.

#### Transformación de los Datos

En esta subsección se arreglan, limpian y transforman el dataset de productos.

In [767]:
# Arreglo a la ADRESS_STATE_ID: para algunos productos el ADRESS_STATE_ID hay que arreglar su dato
# Se uso el ITEM_PERMALINK para entender cual era el verdadero dato de ADRESS_STATE_ID
cambios_de_direccion = {
                        'TUNPUEJPR1gxMDljZA': 'CO-DC', 
                        'TUNPUEFOVGFiZWI3': 'CO-ANT ',
                        'TUNPUENVTmE3NmQ4': 'CO-CUN',
                        'TUNPUFZBTGExNmNjNg': 'CO-VAC',
                        'TUNPUFJJU2ExMWIyYg': 'CO-RIS',
                        'TUNPUEFUTG9mNDk5': 'CO-ATL',
                        'TUNPUFRPTGExNGZkNA': 'CO-TOL',
                        'TUNPUFNBTnJlMjMw': 'CO-SAN',
                        'TUNPUEJPTHI1Mzlk': 'CO-BOL',
                        'TUNPUE1BR2FiZjQ0': 'CO-MAG',
                        'TUNPUENBTHNjODY4': 'CO-CAL',
                        'TUNPUE1FVGExNzFjNQ': 'CO-MET',
                        'TUNPUFFVSW9kYmZm': 'CO-QUI',
                        'TUNPUE5PUnIxNDkyZg': 'CO-NSA', 
                        'TUNPUEFSQ2E4Zjc3': 'CO-SAP',
                        'TUNPUEJPWWE4YzMz': 'CO-BOY',
                        'TUNPUENBVWExM2Q1NQ': 'CO-CAU',
                        'TUNPUENPUmFkZGIw': 'CO-COR',
                        'TUNPUEdVQWExOTYx': 'CO-LAG',
                        'TUNPUE5BUm8xYzk4': 'CO-NAR', 
                        'TUNPUFNVQ2U4ZWQ0': 'CO-SUC',
                        'TUNPUENBU2U2OWIy': 'CO-CAS',
                        'TUNPUENFU3IxODA4Mg': 'CO-CES',
                        'TUNPUEFNQXMxMzQ2YQ': 'CO-AMA'
                       }

# Arregla un error que se detecto en las direcciones
products.loc[products['ADRESS_STATE_ID'].isin(cambios_de_direccion.keys()), 'ADRESS_STATE_ID'] = \
products.loc[products['ADRESS_STATE_ID'].isin(cambios_de_direccion.keys()), 'ADRESS_STATE_ID'].map(cambios_de_direccion)

In [768]:
## Ajuste a la condicion del item

# Se cambia nan por not_specified
products.loc[products['ITEM_CONDITION'].isna(), 'ITEM_CONDITION'] = 'not_specified'

# Ajustes al precio 

# Se quitan productos que no tienen precio (generalmente precio a convenir)
# Es decir este analisis aplica para productos con precio fijo
products = products.loc[~products['ITEM_PRICE'].isna()]
products.reset_index(drop=True, inplace=True)

# Se encontro en google la tasa de cambio de dolar a cop el dia 7 de marzo (cuando se descargaron los datos)
exchange_rate_usd_to_cop = 3142.99 
products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_PRICE'] *= exchange_rate_usd_to_cop
products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_CURRENCY_ID'] = 'COP'

# Cambia ITEM_ORIGINAL_PRICE por el ITEM_PRICE como se indica en el documento
products.loc[products['ITEM_ORIGINAL_PRICE'].isna(), 'ITEM_ORIGINAL_PRICE'] = \
products.loc[products['ITEM_ORIGINAL_PRICE'].isna(), 'ITEM_PRICE']

# Crea las variables de descuento absoluto y la tasa
products['ITEM_DISCOUNT_VALUE'] = products['ITEM_ORIGINAL_PRICE'] - products['ITEM_PRICE']
products['ITEM_DISCOUNT_RATE'] = products['ITEM_DISCOUNT_VALUE']/products['ITEM_ORIGINAL_PRICE']

# Crea la variable de venta total: This is an approximation
products['ITEM_SELL_TOTAL'] = products['ITEM_PRICE']*products['ITEM_SOLD_QUANTITY']

In [769]:
## Ajuste a variables del vendedor

# Cambio de nan to not_specified para SELLER_STATUS
products.loc[products['SELLER_STATUS'].isna(), 'SELLER_STATUS'] = 'not_specified'
products.loc[products['SELLER_LEVEL_ID'].isna(), 'SELLER_LEVEL_ID'] = 'not_specified'

# Cambio nan to 0 para SELLER_METRICS_SALES_PERIOD
products.loc[products['SELLER_METRICS_SALES_PERIOD'].isna(), 'SELLER_METRICS_SALES_COMPLETED'] = 0
products.loc[products['SELLER_METRICS_SALES_PERIOD'].isna(), 'SELLER_METRICS_SALES_PERIOD'] = '60 days'
products.loc[products['SELLER_METRICS_SALES_PERIOD'] == '60 months', 'SELLER_METRICS_SALES_COMPLETED'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_SALES_PERIOD'] == '3 months', 'SELLER_METRICS_SALES_COMPLETED'] *= (60/90)
products.loc[products['SELLER_METRICS_SALES_PERIOD'] == '365 days', 'SELLER_METRICS_SALES_COMPLETED'] *= (60/365)
products['SELLER_METRICS_SALES_PERIOD'] = '60 days'

# Estandarizacion de las metricas absolutas del vendedor (por periodo de tiempo)

# Cancelaciones 
products.loc[products['SELLER_METRICS_CANCELLATIONS_PERIOD'] == '60 months', 'SELLER_METRICS_CANCELLATIONS_VALUE'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_CANCELLATIONS_PERIOD'] == '3 months', 'SELLER_METRICS_CANCELLATIONS_VALUE'] *= (60/90)
products.loc[products['SELLER_METRICS_CANCELLATIONS_PERIOD'] == '365 days', 'SELLER_METRICS_CANCELLATIONS_VALUE'] *= (60/365)
products['SELLER_METRICS_CANCELLATIONS_PERIOD'] = '60 days'

# Quejas 
products.loc[products['SELLER_METRICS_CLAIMS_PERIOD'] == '60 months', 'SELLER_METRICS_CLAIMS_VALUE'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_CLAIMS_PERIOD'] == '3 months', 'SELLER_METRICS_CLAIMS_VALUE'] *= (60/90)
products.loc[products['SELLER_METRICS_CLAIMS_PERIOD'] == '365 days', 'SELLER_METRICS_CLAIMS_VALUE'] *= (60/365)
products['SELLER_METRICS_CLAIMS_PERIOD'] = '60 days'

# Demoras 
products.loc[products['SELLER_METRICS_DELAY_PERIOD'] == '60 months', 'SELLER_METRICS_DELAY_VALUE'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_DELAY_PERIOD'] == '3 months', 'SELLER_METRICS_DELAY_VALUE'] *= (60/90)
products.loc[products['SELLER_METRICS_DELAY_PERIOD'] == '365 days', 'SELLER_METRICS_DELAY_VALUE'] *= (60/365)
products['SELLER_METRICS_DELAY_PERIOD'] = '60 days'

## Ajustes a variables del shipping/envio

# Cambio de nan to not_specified para SHIPPING_LOGISTIC_TYPE
products.loc[products['SHIPPING_LOGISTIC_TYPE'].isna(), 'SHIPPING_LOGISTIC_TYPE'] = 'not_specified'

In [770]:
# Encuentra las categorias cuyas cantidades vendidas son 0 y las remueve
sold_quantity_per_category = products.groupby('SEARCH_CATEGORY_ID')['ITEM_SELL_TOTAL'].sum().sort_values(ascending=False)
categories_zero_sold_quantity = sold_quantity_per_category.loc[sold_quantity_per_category == 0].index.tolist()
products = products.loc[~products['SEARCH_CATEGORY_ID'].isin(categories_zero_sold_quantity)]

# Se remueven los productos cuya cantidad vendida es de 5000
products = products.loc[products['ITEM_SOLD_QUANTITY'] < 5000]
products = products.loc[~products['ITEM_DOMAIN_ID'].isna()]

# Imprime las dimensiones del dataset de products final
print('LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: {}'.format(products.shape))

LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: (13043, 53)


### Selección de Predictores

In [771]:
# Variables de item y search  
item_vars = [
    'ITEM_PRICE',
    'ITEM_AVAILABLE_QUANTITY',
    'ITEM_SOLD_QUANTITY', # Variable de respuesta
    'ITEM_LISTING_TYPE_ID',
    'ITEM_CONDITION',
    'ITEM_DOMAIN_ID',
    'ITEM_ACCEPTS_MERCADOPAGO',
    'ITEM_ORIGINAL_PRICE',
    'ITEM_USE_THUMBNAIL_ID',
    'SEARCH_CATEGORY_ID', # La categoria del busqueda del item 
    'SEARCH_OFFSET', # El offset de busqueda que en el que se obtuvo el item
#     'ITEM_DISCOUNT_VALUE', # Se omite ya que es una combinacion lineal de ITEM_PRICE y ITEM_ORIGINAL_PRICE
    'ITEM_DISCOUNT_RATE',
]

# Variables del seller y el shipping
seller_vars = [
    'SELLER_REP_TRANSACTIONS_TOTAL',
    'SELLER_REP_TRANSACTIONS_CANCELED',
    'SELLER_REP_RATING_NEG',
    'SELLER_REP_RATING_POS',
    'SELLER_REP_RATING_NEU',
    'SELLER_TRANSACTIONS_COMPLETED',
    'SELLER_METRICS_CLAIMS_RATE',
    'SELLER_METRICS_CLAIMS_VALUE',
    'SELLER_METRICS_DELAY_RATE',
    'SELLER_METRICS_DELAY_VALUE', 
    'SELLER_METRICS_SALES_COMPLETED',
    'SELLER_METRICS_CANCELLATIONS_RATE',
    'SELLER_METRICS_CANCELLATIONS_VALUE', 
    'SELLER_LEVEL_ID',
    'SHIPPING_FREE',
    'SHIPPING_LOGISTIC_TYPE', 
    'SHIPPING_PICK_UP',
    'ADRESS_STATE_ID', 
]

# Seleeciona unicamente columnas anteriormente definidas
products_selected = products[item_vars + seller_vars]
products_selected.reset_index(drop=True, inplace=True)
print('LAS DIMENSIONES DE LA DATA DE PRODUCTOS SON: {}'.format(products_selected.shape))

LAS DIMENSIONES DE LA DATA DE PRODUCTOS SON: (13043, 30)


### Estandarización, OneHotEncoding y Train-Test Split

In [772]:
# Crea un dataframe de variables numericas y otro de categoricas
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
products_numeric = products_selected.select_dtypes(include=numerics)

categorical = ['object']
products_categorical = products_selected.select_dtypes(include=categorical)

# Cambia las variables 
binary = ['bool']
products_binary = products_selected.select_dtypes(include=binary)

# Revisa que todas las variables de products_selected esten en products_numeric, products_binary y products_categorical
assert(products_selected.shape[1] == (products_categorical.shape[1]+products_numeric.shape[1]+products_binary.shape[1]))

# Imprime las dimensiones de cada base
print('LAS DIMENSIONES DE LA DATA NUMERICA DE PRODUCTOS SON: {}'.format(products_numeric.shape))
print('LAS DIMENSIONES DE LA DATA CATEGORICA DE PRODUCTOS SON: {}'.format(products_categorical.shape))
print('LAS DIMENSIONES DE LA DATA BINARIA DE PRODUCTOS SON: {}'.format(products_binary.shape))

LAS DIMENSIONES DE LA DATA NUMERICA DE PRODUCTOS SON: (13043, 19)
LAS DIMENSIONES DE LA DATA CATEGORICA DE PRODUCTOS SON: (13043, 7)
LAS DIMENSIONES DE LA DATA BINARIA DE PRODUCTOS SON: (13043, 4)


In [773]:
## Cambio a 1-0 para las binarias
# Cambia las variables binarias True-False por 1-0 
products_binary = products_binary.astype(int)

## OneHotEncoding para las columnas categoricas
# Define el one_hot_encoder
one_hot_encoder = OneHotEncoder(sparse=False)
# Transforma los datos
products_encoded_categoricals = one_hot_encoder.fit_transform(products_categorical)

In [774]:
# Divide los datos en entrenamiento y prueba
Y = products_numeric['ITEM_SOLD_QUANTITY']
X = products_numeric.drop(columns='ITEM_SOLD_QUANTITY')
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3, random_state=1608)
indexes_train = X_train.index.tolist()
indexes_test = X_test.index.tolist()

## Estandarizacion para las numericas
standard_scaler = StandardScaler() 
X_train_std = standard_scaler.fit_transform(X_train)
X_test_std = standard_scaler.transform(X_test)

## Construye la matriz de datos con todos los tipos de data
X_train_total = np.concatenate((X_train_std, products_encoded_categoricals[indexes_train,:],
                                products_binary.loc[indexes_train]), axis=1)
X_test_total = np.concatenate((X_test_std, products_encoded_categoricals[indexes_test,:],
                                products_binary.loc[indexes_test]), axis=1)

## Modelos de Predicción - Sold Quantity

En esta sección se construye y evalua el rendimiento de disintos modelos de Machine Learning para estimar la cantidad vendida de un ítem en función de las variables escogidas previamente.

### XGBoost (Gradient Boosting Trees)

Uno de los algoritmos que escogí para usar es Gradient Boosting Trees a través del framework `XGBoost` conocido por su excelente rendimiento y rapidez. Lo que se hara es buscar los mejores hiper-parámetros por validación cruzada y evaluar el desempeño del modelo.

In [716]:
# Parametros del XGBoost para tunear (se escogio el dart booster para evitar el overfitting)
param_tuning = {
        'learning_rate': [0.10, 0.25],
        'max_depth': [5, 7],
        'min_child_weight': [3, 5],
        'subsample': [1.0],
        'colsample_bytree': [0.50],
        'n_estimators' : [100],
        'objective': ['reg:squarederror'],
        'booster': ['dart'],
        'rate_drop': [0.0, 0.1],
        'skip_drop': [0.1, 0.5]
    }

# Se evalua para todas las combinaciones de hiperparametros cual tiene el menor error de validacion
# usando validacion cruzada para 5-folds

# Modelo XGBRegressor
xgb_model = XGBRegressor()

# Validacion cruzada
k_folds = KFold(n_splits=5, shuffle=True, random_state=1608)
grid_search_cv = GridSearchCV(estimator = xgb_model,
                              param_grid = param_tuning,            
                              cv = k_folds,
                              n_jobs = -1,
                              verbose = 1)
cv_fit = grid_search_cv.fit(X=X_train_total, y=Y_train) # Fit

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.1min finished


In [717]:
# Imprime los mejores parametros bajo el score de cv calculado para cada iteracion
grid_search_cv.best_params_

{'booster': 'dart',
 'colsample_bytree': 0.5,
 'learning_rate': 0.25,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'rate_drop': 0.1,
 'skip_drop': 0.5,
 'subsample': 1.0}

In [742]:
# Se generan las predicciones para el entrenamiento y prueba y se evaluan dos metricas: RMSE y MAE
xgb_best_model = grid_search_cv.best_estimator_
test_predictions = xgb_best_model.predict(X_test_total)
train_predictions = xgb_best_model.predict(X_train_total)

In [743]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 79.35
RMSE DE PRUEBA: 104.87
DIFERENCIA DE: 32.17%


In [744]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 54.74
MAE DE PRUEBA: 72.84
DIFERENCIA DE: 33.07%


In [745]:
values = sorted(Y_train.unique())
test_predictions = [find_nearest(values, p)for p in test_predictions] 
train_predictions = [find_nearest(values, p)for p in train_predictions] 

In [746]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 86.3
RMSE DE PRUEBA: 111.63
DIFERENCIA DE: 29.35%


In [747]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 49.49
MAE DE PRUEBA: 69.39
DIFERENCIA DE: 40.21%


In [750]:
# Se crea un dataframe con los resultados
resultados_train = pd.DataFrame({'TYPE': ['TRAIN']*len(Y_train), 'TRUE': Y_train, 'PREDICTED': train_predictions})
resultados_test = pd.DataFrame({'TYPE': ['TEST']*len(Y_test), 'TRUE': Y_test, 'PREDICTED': test_predictions})
resultados_xgb = pd.concat([resultados_train, resultados_test], axis=0).reset_index(drop=True)
resultados_xgb['ABSOLUTE_ERROR'] = (resultados_xgb['TRUE']-resultados_xgb['PREDICTED']).abs()
resultados_xgb['SQUARE_ERROR'] = (resultados_xgb['TRUE']-resultados_xgb['PREDICTED'])**2

In [758]:
# Resultados por cantidad
resultados_por_cantidad = resultados_xgb.groupby(['TYPE', 'TRUE'])[['ABSOLUTE_ERROR', 'SQUARE_ERROR']].mean().reset_index()

In [759]:
# Imprime los resultados por cantidad en cuanto al MAE y MSE para los datos de prueba
resultados_por_cantidad.loc[resultados_por_cantidad['TYPE']=='TEST']

,TYPE,TRUE,ABSOLUTE_ERROR,SQUARE_ERROR
0,TEST,0,13.389937,509.805031
1,TEST,1,14.776119,718.059701
2,TEST,2,11.974026,381.818182
3,TEST,3,18.562500,864.020833
4,TEST,4,15.784615,673.815385
5,TEST,5,38.708943,3173.508943
6,TEST,6,69.000000,5386.000000
7,TEST,7,52.000000,2704.000000
8,TEST,8,8.000000,64.000000
9,TEST,14,11.000000,121.000000


### NNs (Redes Neuronales)

Uno de los algoritmos que escogí para usar es Redes Neuronales utilizando `Tensorflow` y en particular el modelo de `Keras` dentro de este. 

In [775]:
# Semilla aleatoria fijada 
tf.random.set_seed(200)

# Capa General 1: Capa Convolucional
nn_model = tf.keras.models.Sequential()

# Capa de 10 neuronas con activacion relu y dropout de 0.25
nn_model.add(tf.keras.layers.Dense(10, input_shape=(X_train_total.shape[1],), activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

# Capa de 25 neuronas con activacion relu y dropout de 0.25
nn_model.add(tf.keras.layers.Dense(25, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

# Capa de 25 neuronas con activacion relu y dropout de 0.5
nn_model.add(tf.keras.layers.Dense(25, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.5))

# Capa de 25 neuronas con activacion relu y dropout de 0.25
nn_model.add(tf.keras.layers.Dense(25, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

# Capa de 10 neuronas con activacion relu y dropout de 0.25
nn_model.add(tf.keras.layers.Dense(10, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

# Capa de output 
nn_model.add(tf.keras.layers.Dense(1))

# Se especifica cual es el optimizador (metodo de descenso del gradiente a usar, la función de perdida y las metricas a mostrar)
nn_model.compile(optimizer='adam',loss=tf.keras.losses.MeanSquaredError(), metrics=['mean_absolute_error'])

In [778]:
nn_model.fit(X_train_total, Y_train, epochs=250, batch_size=2000, verbose=1, validation_split=0.1)

Epoch 1/250
5/5 [==============================] - 1s 41ms/step - loss: 52650.9043 - mean_absolute_error: 149.9928 - val_loss: 51905.9492 - val_mean_absolute_error: 148.5737
Epoch 2/250
5/5 [==============================] - 0s 14ms/step - loss: 52538.4714 - mean_absolute_error: 150.0444 - val_loss: 51898.1016 - val_mean_absolute_error: 148.5480
Epoch 3/250
5/5 [==============================] - 0s 14ms/step - loss: 52836.4277 - mean_absolute_error: 150.4700 - val_loss: 51888.7812 - val_mean_absolute_error: 148.5190
Epoch 4/250
5/5 [==============================] - 0s 14ms/step - loss: 52232.1465 - mean_absolute_error: 149.2861 - val_loss: 51876.2031 - val_mean_absolute_error: 148.4797
Epoch 5/250
5/5 [==============================] - 0s 14ms/step - loss: 52723.1159 - mean_absolute_error: 150.4728 - val_loss: 51858.6289 - val_mean_absolute_error: 148.4248
Epoch 6/250
5/5 [==============================] - 0s 14ms/step - loss: 53278.4167 - mean_absolute_error: 151.4291 - val_loss: 518

Epoch 48/250
5/5 [==============================] - 0s 15ms/step - loss: 19323.5752 - mean_absolute_error: 92.7917 - val_loss: 14857.0127 - val_mean_absolute_error: 82.4788
Epoch 49/250
5/5 [==============================] - 0s 15ms/step - loss: 20261.6178 - mean_absolute_error: 95.3989 - val_loss: 14539.6777 - val_mean_absolute_error: 81.8847
Epoch 50/250
5/5 [==============================] - 0s 14ms/step - loss: 19372.7061 - mean_absolute_error: 92.4866 - val_loss: 14461.4805 - val_mean_absolute_error: 81.1582
Epoch 51/250
5/5 [==============================] - 0s 14ms/step - loss: 18724.6152 - mean_absolute_error: 90.8575 - val_loss: 14330.7041 - val_mean_absolute_error: 80.5303
Epoch 52/250
5/5 [==============================] - 0s 14ms/step - loss: 19514.0693 - mean_absolute_error: 92.4588 - val_loss: 14320.1416 - val_mean_absolute_error: 80.1768
Epoch 53/250
5/5 [==============================] - 0s 14ms/step - loss: 18762.3590 - mean_absolute_error: 90.3203 - val_loss: 14121.33

5/5 [==============================] - 0s 14ms/step - loss: 15678.7627 - mean_absolute_error: 82.7190 - val_loss: 12579.1094 - val_mean_absolute_error: 75.5228
Epoch 96/250
5/5 [==============================] - 0s 14ms/step - loss: 15842.5975 - mean_absolute_error: 82.8084 - val_loss: 12447.2617 - val_mean_absolute_error: 75.4633
Epoch 97/250
5/5 [==============================] - 0s 14ms/step - loss: 15953.5368 - mean_absolute_error: 83.9259 - val_loss: 12349.5391 - val_mean_absolute_error: 75.4718
Epoch 98/250
5/5 [==============================] - 0s 14ms/step - loss: 15020.2754 - mean_absolute_error: 81.5637 - val_loss: 12318.5771 - val_mean_absolute_error: 75.5154
Epoch 99/250
5/5 [==============================] - 0s 15ms/step - loss: 15895.1092 - mean_absolute_error: 83.8724 - val_loss: 12291.1826 - val_mean_absolute_error: 75.4052
Epoch 100/250
5/5 [==============================] - 0s 14ms/step - loss: 15291.1104 - mean_absolute_error: 81.7863 - val_loss: 12292.5977 - val_mea

5/5 [==============================] - 0s 14ms/step - loss: 15065.1271 - mean_absolute_error: 81.5731 - val_loss: 11881.4443 - val_mean_absolute_error: 74.5671
Epoch 143/250
5/5 [==============================] - 0s 14ms/step - loss: 15086.1546 - mean_absolute_error: 81.2970 - val_loss: 12266.2559 - val_mean_absolute_error: 74.8880
Epoch 144/250
5/5 [==============================] - 0s 14ms/step - loss: 14713.9147 - mean_absolute_error: 80.4576 - val_loss: 12480.6494 - val_mean_absolute_error: 75.1595
Epoch 145/250
5/5 [==============================] - 0s 14ms/step - loss: 15114.2352 - mean_absolute_error: 81.2054 - val_loss: 12065.9941 - val_mean_absolute_error: 74.7425
Epoch 146/250
5/5 [==============================] - 0s 14ms/step - loss: 14619.5220 - mean_absolute_error: 80.2758 - val_loss: 11969.4932 - val_mean_absolute_error: 74.5837
Epoch 147/250
5/5 [==============================] - 0s 14ms/step - loss: 15277.8239 - mean_absolute_error: 81.7753 - val_loss: 12361.6758 - val

5/5 [==============================] - 0s 14ms/step - loss: 14210.9598 - mean_absolute_error: 78.2462 - val_loss: 13112.3027 - val_mean_absolute_error: 76.4932
Epoch 190/250
5/5 [==============================] - 0s 14ms/step - loss: 14376.5879 - mean_absolute_error: 78.3926 - val_loss: 12255.2324 - val_mean_absolute_error: 75.8233
Epoch 191/250
5/5 [==============================] - 0s 14ms/step - loss: 14725.0335 - mean_absolute_error: 80.8668 - val_loss: 11982.0723 - val_mean_absolute_error: 75.6220
Epoch 192/250
5/5 [==============================] - 0s 14ms/step - loss: 14909.6685 - mean_absolute_error: 81.4685 - val_loss: 12596.3066 - val_mean_absolute_error: 75.9790
Epoch 193/250
5/5 [==============================] - 0s 14ms/step - loss: 14583.4232 - mean_absolute_error: 79.5031 - val_loss: 13198.6416 - val_mean_absolute_error: 76.7447
Epoch 194/250
5/5 [==============================] - 0s 14ms/step - loss: 14844.3389 - mean_absolute_error: 80.7194 - val_loss: 12556.7529 - val

5/5 [==============================] - 0s 14ms/step - loss: 13929.6779 - mean_absolute_error: 79.3360 - val_loss: 12747.7490 - val_mean_absolute_error: 76.2349
Epoch 237/250
5/5 [==============================] - 0s 14ms/step - loss: 14089.8434 - mean_absolute_error: 78.8042 - val_loss: 13000.6133 - val_mean_absolute_error: 76.3482
Epoch 238/250
5/5 [==============================] - 0s 14ms/step - loss: 14153.7446 - mean_absolute_error: 78.7840 - val_loss: 12852.6299 - val_mean_absolute_error: 76.1653
Epoch 239/250
5/5 [==============================] - 0s 14ms/step - loss: 14633.3805 - mean_absolute_error: 79.5342 - val_loss: 12920.6250 - val_mean_absolute_error: 76.3987
Epoch 240/250
5/5 [==============================] - 0s 14ms/step - loss: 14321.6971 - mean_absolute_error: 79.2321 - val_loss: 13153.3994 - val_mean_absolute_error: 76.9001
Epoch 241/250
5/5 [==============================] - 0s 14ms/step - loss: 13896.9940 - mean_absolute_error: 77.7874 - val_loss: 12765.0811 - val

In [779]:
# Se generan las predicciones para el entrenamiento y prueba y se evaluan dos metricas: RMSE y MAE
test_predictions = nn_model.predict(X_test_total)
train_predictions = nn_model.predict(X_train_total)

In [780]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 101.38
RMSE DE PRUEBA: 112.71
DIFERENCIA DE: 11.17%


In [781]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 67.57
MAE DE PRUEBA: 75.05
DIFERENCIA DE: 11.07%


In [782]:
values = sorted(Y_train.unique())
test_predictions = [find_nearest(values, p)for p in test_predictions] 
train_predictions = [find_nearest(values, p)for p in train_predictions] 

In [783]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 110.47
RMSE DE PRUEBA: 118.93
DIFERENCIA DE: 7.66%


In [784]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 68.29
MAE DE PRUEBA: 74.07
DIFERENCIA DE: 8.47%


In [785]:
# Se crea un dataframe con los resultados
resultados_train = pd.DataFrame({'TYPE': ['TRAIN']*len(Y_train), 'TRUE': Y_train, 'PREDICTED': train_predictions})
resultados_test = pd.DataFrame({'TYPE': ['TEST']*len(Y_test), 'TRUE': Y_test, 'PREDICTED': test_predictions})
resultados_nn = pd.concat([resultados_train, resultados_test], axis=0).reset_index(drop=True)
resultados_nn['ABSOLUTE_ERROR'] = (resultados_nn['TRUE']-resultados_nn['PREDICTED']).abs()
resultados_nn['SQUARE_ERROR'] = (resultados_nn['TRUE']-resultados_nn['PREDICTED'])**2

In [786]:
# Resultados por cantidad
resultados_por_cantidad = resultados_nn.groupby(['TYPE', 'TRUE'])[['ABSOLUTE_ERROR', 'SQUARE_ERROR']].mean().reset_index()

In [787]:
# Imprime los resultados por cantidad en cuanto al MAE y MSE para los datos de prueba
resultados_por_cantidad.loc[resultados_por_cantidad['TYPE']=='TEST']

,TYPE,TRUE,ABSOLUTE_ERROR,SQUARE_ERROR
0,TEST,0,32.572327,1091.150943
1,TEST,1,31.567164,1077.268657
2,TEST,2,30.506494,1005.077922
3,TEST,3,32.562500,1650.562500
4,TEST,4,27.738462,782.169231
5,TEST,5,37.769106,2408.884553
6,TEST,6,94.000000,8836.000000
7,TEST,7,93.000000,8649.000000
8,TEST,8,27.000000,729.000000
9,TEST,14,45.000000,2025.000000


### Random Forest (Bosques Aleatorios)

Uno de los algoritmos que escogí para usar es Bosques Aleatorios utilizando `ScikitLearn`. 

In [791]:
# Diccionario de hiperparametros a tunear
param_tuning = {'n_estimators': [100],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [5, 7, 10],
               'bootstrap': [False]}

# Se evalua para todas las combinaciones de hiperparametros cual tiene el menor error de validacion
# usando validacion cruzada para 5-folds

# Modelo RFRegressor
rf_model = RandomForestRegressor(random_state=1608)

# Validacion cruzada
k_folds = KFold(n_splits=5, shuffle=True, random_state=1608)
grid_search_cv = GridSearchCV(estimator = rf_model,
                              param_grid = param_tuning,            
                              cv = k_folds,
                              n_jobs = -1,
                              verbose = 1)
cv_fit = grid_search_cv.fit(X=X_train_total, y=Y_train) # Fit

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:   19.6s remaining:   19.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.8min finished


In [792]:
# Imprime los mejores parametros bajo el score de cv calculado para cada iteracion
grid_search_cv.best_params_

{'bootstrap': False,
 'max_depth': 7,
 'max_features': 'auto',
 'n_estimators': 100}

In [793]:
# Se generan las predicciones para el entrenamiento y prueba y se evaluan dos metricas: RMSE y MAE
rf_best_model = grid_search_cv.best_estimator_
test_predictions = rf_best_model.predict(X_test_total)
train_predictions = rf_best_model.predict(X_train_total)

In [794]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 132.08
RMSE DE PRUEBA: 140.36
DIFERENCIA DE: 6.27%


In [795]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 95.76
MAE DE PRUEBA: 102.97
DIFERENCIA DE: 7.54%


In [796]:
values = sorted(Y_train.unique())
test_predictions = [find_nearest(values, p)for p in test_predictions] 
train_predictions = [find_nearest(values, p)for p in train_predictions] 

In [797]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 134.35
RMSE DE PRUEBA: 143.11
DIFERENCIA DE: 6.52%


In [798]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 93.35
MAE DE PRUEBA: 101.16
DIFERENCIA DE: 8.38%


In [799]:
# Se crea un dataframe con los resultados
resultados_train = pd.DataFrame({'TYPE': ['TRAIN']*len(Y_train), 'TRUE': Y_train, 'PREDICTED': train_predictions})
resultados_test = pd.DataFrame({'TYPE': ['TEST']*len(Y_test), 'TRUE': Y_test, 'PREDICTED': test_predictions})
resultados_rf = pd.concat([resultados_train, resultados_test], axis=0).reset_index(drop=True)
resultados_rf['ABSOLUTE_ERROR'] = (resultados_rf['TRUE']-resultados_rf['PREDICTED']).abs()
resultados_rf['SQUARE_ERROR'] = (resultados_rf['TRUE']-resultados_rf['PREDICTED'])**2

In [800]:
# Resultados por cantidad
resultados_por_cantidad = resultados_rf.groupby(['TYPE', 'TRUE'])[['ABSOLUTE_ERROR', 'SQUARE_ERROR']].mean().reset_index()

In [801]:
# Imprime los resultados por cantidad en cuanto al MAE y MSE para los datos de prueba
resultados_por_cantidad.loc[resultados_por_cantidad['TYPE']=='TEST']

,TYPE,TRUE,ABSOLUTE_ERROR,SQUARE_ERROR
0,TEST,0,47.169811,4231.371069
1,TEST,1,55.626866,5202.582090
2,TEST,2,49.532468,3982.519481
3,TEST,3,50.541667,5144.000000
4,TEST,4,48.323077,4110.876923
5,TEST,5,81.377236,12102.037398
6,TEST,6,148.500000,31172.500000
7,TEST,7,28.000000,784.000000
8,TEST,8,192.000000,36864.000000
9,TEST,14,86.000000,7396.000000


### Stacked Model

Una idea final que tuve para lograr el mejor modelo posible dentro del tiempo que tenía para experimentar con los datos fue hacer un modelo que a partir de las predicciones de los tres modelos presentados anteriormente.

In [807]:
# Construccion de dataset de predicciones
resultados = resultados_xgb[['TYPE', 'TRUE', 'PREDICTED']].rename(columns={'PREDICTED': 'PREDICTED_XGB'})
resultados['PREDICTED_NN'] = resultados_nn['PREDICTED']
resultados['PREDICTED_RF'] = resultados_rf['PREDICTED']

# Separa en train y test
X_train_stack = resultados.loc[resultados['TYPE']=='TRAIN'].drop(columns=['TYPE', 'TRUE'])
X_test_stack = resultados.loc[resultados['TYPE']=='TEST'].drop(columns=['TYPE', 'TRUE'])
Y_train_stack = resultados.loc[resultados['TYPE']=='TRAIN', 'TRUE'] 
Y_test_stack = resultados.loc[resultados['TYPE']=='TEST', 'TRUE']

In [833]:
# Diccionario de hiperparametros a tunear
param_tuning = {'n_estimators': [100, 1000],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [3, 5, 7, 10],
               'bootstrap': [False]}

# Se evalua para todas las combinaciones de hiperparametros cual tiene el menor error de validacion
# usando validacion cruzada para 5-folds

# Modelo RFRegressor
rf_stack_model = RandomForestRegressor(random_state=1608)

# Validacion cruzada
k_folds = KFold(n_splits=5, shuffle=True, random_state=1608)
grid_search_cv = GridSearchCV(estimator = rf_model,
                              param_grid = param_tuning,            
                              cv = k_folds,
                              n_jobs = -1,
                              verbose = 1)
cv_fit = grid_search_cv.fit(X=X_train_stack, y=Y_train_stack) # Fit

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    9.8s finished


In [834]:
# Imprime los mejores parametros bajo el score de cv calculado para cada iteracion
grid_search_cv.best_params_

{'bootstrap': False,
 'max_depth': 7,
 'max_features': 'sqrt',
 'n_estimators': 1000}

In [835]:
# Se generan las predicciones para el entrenamiento y prueba y se evaluan dos metricas: RMSE y MAE
rf_stack_best_model = grid_search_cv.best_estimator_
test_predictions = rf_stack_best_model.predict(X_test_stack)
train_predictions = rf_stack_best_model.predict(X_train_stack)

In [836]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 75.68
RMSE DE PRUEBA: 104.24
DIFERENCIA DE: 37.73%


In [837]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 47.94
MAE DE PRUEBA: 66.9
DIFERENCIA DE: 39.54%


In [838]:
values = sorted(Y_train.unique())
test_predictions = [find_nearest(values, p)for p in test_predictions] 
train_predictions = [find_nearest(values, p)for p in train_predictions] 

In [839]:
# Comparacion RSME train y test
rmse_train = np.sqrt(mean_squared_error(y_pred=train_predictions, y_true=Y_train))
rmse_test = np.sqrt(mean_squared_error(y_pred=test_predictions, y_true=Y_test))
print('RMSE DE ENTRENAMIENTO: {}'.format(np.round(rmse_train,2)))
print('RMSE DE PRUEBA: {}'.format(np.round(rmse_test,2)))
diferencia_porcentual = np.round(100*(rmse_test-rmse_train)/rmse_train,2)
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

RMSE DE ENTRENAMIENTO: 82.61
RMSE DE PRUEBA: 109.4
DIFERENCIA DE: 32.43%


In [840]:
# Comparacion MAE train y test
mae_train = mean_absolute_error(y_pred=train_predictions, y_true=Y_train)
mae_test = mean_absolute_error(y_pred=test_predictions, y_true=Y_test)
diferencia_porcentual = np.round(100*(mae_test-mae_train)/mae_train,2)
print('MAE DE ENTRENAMIENTO: {}'.format(np.round(mae_train,2)))
print('MAE DE PRUEBA: {}'.format(np.round(mae_test,2)))
print('DIFERENCIA DE: {}%'.format(diferencia_porcentual))

MAE DE ENTRENAMIENTO: 44.36
MAE DE PRUEBA: 64.12
DIFERENCIA DE: 44.54%


In [841]:
# Se crea un dataframe con los resultados
resultados_train = pd.DataFrame({'TYPE': ['TRAIN']*len(Y_train), 'TRUE': Y_train, 'PREDICTED': train_predictions})
resultados_test = pd.DataFrame({'TYPE': ['TEST']*len(Y_test), 'TRUE': Y_test, 'PREDICTED': test_predictions})
resultados_rf = pd.concat([resultados_train, resultados_test], axis=0).reset_index(drop=True)
resultados_rf['ABSOLUTE_ERROR'] = (resultados_rf['TRUE']-resultados_rf['PREDICTED']).abs()
resultados_rf['SQUARE_ERROR'] = (resultados_rf['TRUE']-resultados_rf['PREDICTED'])**2

In [842]:
# Resultados por cantidad
resultados_por_cantidad = resultados_rf.groupby(['TYPE', 'TRUE'])[['ABSOLUTE_ERROR', 'SQUARE_ERROR']].mean().reset_index()

In [843]:
# Imprime los resultados por cantidad en cuanto al MAE y MSE para los datos de prueba
resultados_por_cantidad.loc[resultados_por_cantidad['TYPE']=='TEST']

,TYPE,TRUE,ABSOLUTE_ERROR,SQUARE_ERROR
0,TEST,0,9.817610,136.798742
1,TEST,1,11.373134,238.656716
2,TEST,2,9.103896,154.636364
3,TEST,3,11.687500,403.187500
4,TEST,4,8.000000,142.584615
5,TEST,5,25.557724,2007.593496
6,TEST,6,48.500000,2372.500000
7,TEST,7,52.000000,2704.000000
8,TEST,8,17.000000,289.000000
9,TEST,14,21.000000,441.000000


## Alternativa: Modelos de Clasificación

Una alternativa que se propone es trabajar con rangos de `ITEM_SOLD_QUANTITY`. Se usarán para el siguiente análisis los siguientes rangos (que serán tomados como clases):

1. 0-50: De 0 a 50 cantdiades vendidas.

1. 51-100: De 51 a 100 cantdiades vendidas.

1. 101-150: De 101 a 150 cantdiades vendidas.

1. 151-200: De 151 a 200 cantdiades vendidas.

1. 201-250: De 201 a 250 cantdiades vendidas.

1. 251-500: De 251 a 500 cantdiades vendidas.

### Definición de los Rangos de Cantidades Vendidas

Se crea una variable en el dataset de products para definir los rangos/clases de cantidades vendidas.

In [658]:
# Se define la variables de rangos
products['ITEM_RANGE_SOLD_QUANTITY'] = ''
products.loc[(products['ITEM_SOLD_QUANTITY'] >= 0) &
             (products['ITEM_SOLD_QUANTITY'] <= 50), 'ITEM_RANGE_SOLD_QUANTITY'] = '0-50'
products.loc[(products['ITEM_SOLD_QUANTITY'] >= 51) &
             (products['ITEM_SOLD_QUANTITY'] <= 100), 'ITEM_RANGE_SOLD_QUANTITY'] = '51-100'
products.loc[(products['ITEM_SOLD_QUANTITY'] >= 101) &
             (products['ITEM_SOLD_QUANTITY'] <= 150), 'ITEM_RANGE_SOLD_QUANTITY'] = '101-150'
products.loc[(products['ITEM_SOLD_QUANTITY'] >= 151) &
             (products['ITEM_SOLD_QUANTITY'] <= 200), 'ITEM_RANGE_SOLD_QUANTITY'] = '151-200'
products.loc[(products['ITEM_SOLD_QUANTITY'] >= 201) &
             (products['ITEM_SOLD_QUANTITY'] <= 250), 'ITEM_RANGE_SOLD_QUANTITY'] = '201-250'
products.loc[(products['ITEM_SOLD_QUANTITY'] >= 251) &
             (products['ITEM_SOLD_QUANTITY'] <= 500), 'ITEM_RANGE_SOLD_QUANTITY'] = '251-500'

array(['0-50', '201-250', '251-500', '101-150', '51-100', '151-200'],
      dtype=object)

In [659]:
# Divide los datos en entrenamiento y prueba
Y = products['ITEM_RANGE_SOLD_QUANTITY']
X = products_numeric.drop(columns='ITEM_SOLD_QUANTITY')
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3, random_state=1608)
indexes_train = X_train.index.tolist()
indexes_test = X_test.index.tolist()

## Estandarizacion para las numericas
standard_scaler = StandardScaler() 
X_train_std = standard_scaler.fit_transform(X_train)
X_test_std = standard_scaler.transform(X_test)

## Construye la matriz de datos con todos los tipos de data
X_train_total = np.concatenate((X_train_std, products_encoded_categoricals[indexes_train,:],
                                products_binary.loc[indexes_train]), axis=1)
X_test_total = np.concatenate((X_test_std, products_encoded_categoricals[indexes_test,:],
                                products_binary.loc[indexes_test]), axis=1)

In [678]:
# Cambio de variable respuesta a matriz de dummies

## OneHotEncoding para las columnas categoricas
# Define el one_hot_encoder
one_hot_encoder_class = OneHotEncoder(sparse=False)
# Transforma los datos
Y_train_matriz = one_hot_encoder_class.fit_transform(np.array(Y_train).reshape(Y_train.shape[0], 1))
Y_test_matriz = one_hot_encoder_class.transform(np.array(Y_test).reshape(Y_test.shape[0], 1))

### Neural Network

Se construye una red neuronal y se evalua su rendimiento.

In [702]:
# Semilla aleatoria fijada 
tf.random.set_seed(1608)

# Capa General 1: Capa Convolucional
nn_model = tf.keras.models.Sequential()


nn_model.add(tf.keras.layers.Dense(10, input_shape=(X_train_total.shape[1],), activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

nn_model.add(tf.keras.layers.Dense(25, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

nn_model.add(tf.keras.layers.Dense(25, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.5))

nn_model.add(tf.keras.layers.Dense(25, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))

nn_model.add(tf.keras.layers.Dense(10, activation='relu'))
nn_model.add(tf.keras.layers.Dropout(0.25))
          
nn_model.add(tf.keras.layers.Dense(products['ITEM_RANGE_SOLD_QUANTITY'].nunique()))
nn_model.add(tf.keras.layers.Activation('softmax'))

# Se especifica cual es el optimizador (metodo de descenso del gradiente a usar, la función de perdida y las metricas a mostrar)
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [700]:
# Summary de la red neuronal usada
nn_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_121 (Dense)            (None, 10)                11650     
_________________________________________________________________
dropout_101 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_122 (Dense)            (None, 25)                275       
_________________________________________________________________
dropout_102 (Dropout)        (None, 25)                0         
_________________________________________________________________
dense_123 (Dense)            (None, 25)                650       
_________________________________________________________________
dropout_103 (Dropout)        (None, 25)                0         
_________________________________________________________________
dense_124 (Dense)            (None, 25)              

In [703]:
# Fitting del modelo
nn_model.fit(X_train_total, Y_train_matriz, epochs=250, batch_size=2000, verbose=1, validation_split=0.1)

Epoch 1/250
5/5 [==============================] - 1s 44ms/step - loss: 1.8052 - accuracy: 0.1133 - val_loss: 1.7704 - val_accuracy: 0.3527
Epoch 2/250
5/5 [==============================] - 0s 14ms/step - loss: 1.7785 - accuracy: 0.2191 - val_loss: 1.7550 - val_accuracy: 0.5027
Epoch 3/250
5/5 [==============================] - 0s 14ms/step - loss: 1.7609 - accuracy: 0.3154 - val_loss: 1.7379 - val_accuracy: 0.5126
Epoch 4/250
5/5 [==============================] - 0s 14ms/step - loss: 1.7385 - accuracy: 0.3736 - val_loss: 1.7147 - val_accuracy: 0.5126
Epoch 5/250
5/5 [==============================] - 0s 14ms/step - loss: 1.7178 - accuracy: 0.3896 - val_loss: 1.6849 - val_accuracy: 0.5126
Epoch 6/250
5/5 [==============================] - 0s 14ms/step - loss: 1.6905 - accuracy: 0.4095 - val_loss: 1.6496 - val_accuracy: 0.5126
Epoch 7/250
5/5 [==============================] - 0s 14ms/step - loss: 1.6670 - accuracy: 0.4137 - val_loss: 1.6126 - val_accuracy: 0.5126
Epoch 8/250
5/5 [===

5/5 [==============================] - 0s 15ms/step - loss: 1.1739 - accuracy: 0.5116 - val_loss: 1.2120 - val_accuracy: 0.5367
Epoch 60/250
5/5 [==============================] - 0s 15ms/step - loss: 1.1685 - accuracy: 0.5103 - val_loss: 1.2191 - val_accuracy: 0.5509
Epoch 61/250
5/5 [==============================] - 0s 14ms/step - loss: 1.1674 - accuracy: 0.5247 - val_loss: 1.2195 - val_accuracy: 0.5564
Epoch 62/250
5/5 [==============================] - 0s 14ms/step - loss: 1.1623 - accuracy: 0.5333 - val_loss: 1.2305 - val_accuracy: 0.5564
Epoch 63/250
5/5 [==============================] - 0s 14ms/step - loss: 1.1532 - accuracy: 0.5229 - val_loss: 1.2198 - val_accuracy: 0.5564
Epoch 64/250
5/5 [==============================] - 0s 14ms/step - loss: 1.1486 - accuracy: 0.5410 - val_loss: 1.2104 - val_accuracy: 0.5597
Epoch 65/250
5/5 [==============================] - 0s 15ms/step - loss: 1.1552 - accuracy: 0.5429 - val_loss: 1.2181 - val_accuracy: 0.5608
Epoch 66/250
5/5 [========

5/5 [==============================] - 0s 15ms/step - loss: 1.0335 - accuracy: 0.6196 - val_loss: 1.2254 - val_accuracy: 0.6101
Epoch 118/250
5/5 [==============================] - 0s 15ms/step - loss: 1.0294 - accuracy: 0.6182 - val_loss: 1.2228 - val_accuracy: 0.6046
Epoch 119/250
5/5 [==============================] - 0s 15ms/step - loss: 1.0405 - accuracy: 0.6112 - val_loss: 1.2213 - val_accuracy: 0.6068
Epoch 120/250
5/5 [==============================] - 0s 15ms/step - loss: 1.0417 - accuracy: 0.6131 - val_loss: 1.2172 - val_accuracy: 0.6013
Epoch 121/250
5/5 [==============================] - 0s 15ms/step - loss: 1.0358 - accuracy: 0.6169 - val_loss: 1.2235 - val_accuracy: 0.6057
Epoch 122/250
5/5 [==============================] - 0s 14ms/step - loss: 1.0377 - accuracy: 0.6145 - val_loss: 1.2207 - val_accuracy: 0.6046
Epoch 123/250
5/5 [==============================] - 0s 15ms/step - loss: 1.0235 - accuracy: 0.6111 - val_loss: 1.2340 - val_accuracy: 0.6079
Epoch 124/250
5/5 [=

Epoch 175/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9767 - accuracy: 0.6307 - val_loss: 1.2256 - val_accuracy: 0.6068
Epoch 176/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9784 - accuracy: 0.6333 - val_loss: 1.2168 - val_accuracy: 0.6145
Epoch 177/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9766 - accuracy: 0.6250 - val_loss: 1.2312 - val_accuracy: 0.6112
Epoch 178/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9707 - accuracy: 0.6343 - val_loss: 1.2185 - val_accuracy: 0.6123
Epoch 179/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9716 - accuracy: 0.6271 - val_loss: 1.2110 - val_accuracy: 0.6199
Epoch 180/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9828 - accuracy: 0.6249 - val_loss: 1.2168 - val_accuracy: 0.6177
Epoch 181/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9755 - accuracy: 0.6275 - val_loss: 1.2313 - val_accuracy: 0.6188
Epoch 

Epoch 233/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9624 - accuracy: 0.6396 - val_loss: 1.2315 - val_accuracy: 0.6265
Epoch 234/250
5/5 [==============================] - 0s 15ms/step - loss: 0.9535 - accuracy: 0.6434 - val_loss: 1.2354 - val_accuracy: 0.6276
Epoch 235/250
5/5 [==============================] - 0s 14ms/step - loss: 0.9524 - accuracy: 0.6395 - val_loss: 1.2148 - val_accuracy: 0.6287
Epoch 236/250
5/5 [==============================] - 0s 14ms/step - loss: 0.9508 - accuracy: 0.6452 - val_loss: 1.2088 - val_accuracy: 0.6342
Epoch 237/250
5/5 [==============================] - 0s 14ms/step - loss: 0.9545 - accuracy: 0.6389 - val_loss: 1.2376 - val_accuracy: 0.6254
Epoch 238/250
5/5 [==============================] - 0s 14ms/step - loss: 0.9560 - accuracy: 0.6442 - val_loss: 1.2414 - val_accuracy: 0.6243
Epoch 239/250
5/5 [==============================] - 0s 14ms/step - loss: 0.9441 - accuracy: 0.6444 - val_loss: 1.2421 - val_accuracy: 0.6375
Epoch 

## Conclusiones y Reflexiones

Lo primero que se debe resaltar es la dificultad tan fuerte en generar un modelo que estime cuando se tiene una variedad tan diferente de ítems, vendedores y demás, sin mencionar que el conjunto de datos no es más que una muestra (y no comprende el total de ítems de [Mercado Libre Colombia](https://www.mercadolibre.com.co/)). 

El modelo construido usando el concepto de stack/apilar efectivamente resulta ser el mejor cuando se examinan las métricas (de prueba) de RMSE y MAE, lo cual para mí tiene mucho sentido pues junta y combina la __información__ (las predicciones) de cada uno de los modelos individuales. 

La cantidad vendida realmente corresponde a un set de rangos establecidos [aquí](https://developers.mercadolibre.com.ar/en_us/items-and-searches) y además algunos ítems tienen valores que no se presentan en el cuadro que muestra los valores de `sold_quantity` y por lo tanto, hay que añadir que por eso hay un poco de ruído en la variable de respuesta.

Veo un potencial en construir algunas variables (feature engineering) con un poco más de tiempo e imaginación, además de usar tanto el texto del ítem como la imagen de este que efectivamente se puede obtener con la información del conjunto de datos (ver el código anexo). Sin embargo, el uso de imágenes y texto requeriría una muestra más grande de productos (tal vez de otros países también).

`### TO DO: CONCLUSIONES DE MAS RESULTADOS Y LA PARTE DE CLASIFICACION`

In [880]:
# Descarga la imagen con la url de un item
img_url_example = products.loc[1608, 'ITEM_THUMBNAIL']
response = requests.get(img_url_example)

# Guarda la imagen
print('DESCARGADA IMAGEN DE: {}'.format(products.loc[1608, 'ITEM_TITLE'].upper()))
file = open("../Data/Images/EXAMPLE.png", "wb")
file.write(response.content)
file.close()

DESCARGADA IMAGEN DE: LLAVERO LUJO TRENZA CUERO MARCA TOYOTA PRADO TXL FORTU CRUIS


<img src="../Data/Images/EXAMPLE.png " alt="drawing" width="200" title='Ejemplo de la miniatura de un ítem'/>
<center>  <font size="1"> Ejemplo de la miniatura de un ítem del market place de Mercado Libre Colombia</font> </center>

# Fin del Documento